In [5]:
import pandas as pd, pathlib, re

RAW = pathlib.Path("../data_raw/noaa_microplastics_global.csv")

# 1. Carga segura (low_memory=False evita el warning de tipos mezclados)
df = pd.read_csv(RAW, low_memory=False)

# 2. Limpieza de encabezados ----------------------------------------------
def clean(col):
    col = col.encode('utf-8').decode('utf-8-sig')  # elimina BOM si lo hubiera
    col = col.strip()                              # recorta espacios extremos
    col = re.sub(r'\s*\(.*?\)', '', col)           # elimina texto entre paréntesis
    col = re.sub(r'\s+', '_', col)                 # espacios internos → _
    return col

df.columns = [clean(c) for c in df.columns]

# 3. Localizar lat / lon sin asumir un nombre exacto -----------------------
lat_candidates = [c for c in df.columns if re.search(r'(^|_)lat', c, re.I)]
lon_candidates = [c for c in df.columns if re.search(r'(^|_)lon', c, re.I)]

if not lat_candidates or not lon_candidates:
    raise RuntimeError(f"No encuentro columnas lat/lon después de limpiar.\n"
                       f"Encabezados = {df.columns.tolist()[:15]}")

lat_col, lon_col = lat_candidates[0], lon_candidates[0]  # tomamos la 1.ª coincidencia

# Convierte a numérico y elimina filas sin coordenadas
df[lat_col] = pd.to_numeric(df[lat_col], errors='coerce')
df[lon_col] = pd.to_numeric(df[lon_col], errors='coerce')
df = df.dropna(subset=[lat_col, lon_col])

# 4. Filtrar Chile (-56 a -17 lat, -80 a -66 lon) --------------------------
mask = (df[lat_col].between(-56, -17) &
        df[lon_col].between(-80, -66))
df_chile = df[mask]

out = pathlib.Path("../data_raw/microplastics_chile.csv")
df_chile.to_csv(out, index=False)

print(f"Global: {df.shape}  |  Chile: {df_chile.shape}")
print(f"Archivo Chile guardado en: {out.resolve()}")


Global: (20693, 32)  |  Chile: (6, 32)
Archivo Chile guardado en: /home/ricardo/tfm-microplastics/data_raw/microplastics_chile.csv


In [6]:
print(df.isna().sum().sort_values().tail(15))  # últimas 15 columnas con más nulos


Mesh_size                      5858
Region                        11449
Collecting_Time               14882
Volunteers_Number             14882
Standardized_Nurdle_Amount    14882
Subregion                     19101
Sampling_point_on_beach       20273
Body_part_analyzed            20280
Species_scientific_name       20280
Species_class                 20280
Species_common_name           20280
Body_weight                   20318
Total_body_length             20326
Tissue_weight                 20480
Transect_No                   20535
dtype: int64
